<a href="https://colab.research.google.com/github/mebalint/hgt/blob/main/hgt_football_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Predicting football match results with machine learning




### Importing packages

In [4]:
import pandas as pd
#from segedfgek import *
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.model_selection import train_test_split

#from SBS import *
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.gaussian_process.kernels import RBF

#from PlotDecisionRegions import *
from sklearn.decomposition import PCA

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import KernelPCA

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier

from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso, LassoLars, Ridge, BayesianRidge, SGDRegressor, LinearRegression
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor

from sklearn.model_selection import RandomizedSearchCV

# to supress warnings
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1)

### Data preparation
A https://figshare.com/collections/Soccer_match_event_dataset/4415000 címen nyilvánosan elérhető adatból nyertük ki saját számításaink alapján a számunkra fontos mennyiségeket. Az adat az európai top 5 futball bajnokság (angol, német, olasz, spanyol, francia) 2017/18-as szezonjának összes meccséről szól, egy adott meccsen minden labdás eseményről van feljegyzés (pl. passzok, lövések, gólok). Ezen események alapján az egyszerű meccsstatisztikákat (pl. kapura lövés, szabadrúgások) és néhány hálózattudományi mértéket (pl. betweenness centrality) határoztunk meg.

In [27]:
matches=pd.read_csv('https://raw.githubusercontent.com/mebalint/hgt/main/matches_final.csv', sep='\t', index_col=0)
matches=matches.drop(columns=['competitionId', 'roundId', 'match_id', 'draw_odds'])
matches

,home,away,home_goal,away_goal,gameweek,home_corner,away_corner,home_pass,away_pass,home_duel,...,home_possession,away_possession,home_degree,away_degree,home_diameter,away_diameter,home_bet_cent,away_bet_cent,home_entropy,away_entropy
0,Lazio,Inter,2,3,38,10,3,302,492,228,...,0.40,0.60,22.000,37.545,2,2,0.027,0.019,0.072,0.032
1,Sassuolo,AS Roma,0,1,38,5,9,337,500,206,...,0.42,0.58,23.182,36.909,2,2,0.024,0.011,0.066,0.032
2,Cagliari,Atalanta,1,0,38,5,11,232,587,235,...,0.31,0.69,15.545,45.909,2,2,0.037,0.019,0.111,0.027
3,Chievo,Benevento,1,0,38,2,4,198,721,221,...,0.24,0.76,12.818,55.364,2,2,0.042,0.025,0.144,0.021
4,Udinese,Bologna,1,0,38,10,2,327,518,207,...,0.41,0.59,24.636,39.636,3,2,0.033,0.019,0.064,0.032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,Tottenham,Swansea,0,0,5,11,1,753,220,177,...,0.75,0.25,60.636,13.909,2,3,0.013,0.037,0.018,0.130
1822,Tottenham,Burnley,1,1,3,10,7,573,276,179,...,0.66,0.34,44.909,18.545,2,2,0.016,0.026,0.026,0.086
1823,Tottenham,Chelsea,1,2,2,14,3,569,249,223,...,0.68,0.32,43.545,17.091,2,2,0.009,0.030,0.025,0.098
1824,Sevilla,Alaves,1,0,38,8,3,601,361,205,...,0.61,0.39,48.091,25.364,2,3,0.013,0.030,0.024,0.057


In [28]:
matches.columns

Index(['home', 'away', 'home_goal', 'away_goal', 'gameweek', 'home_corner',
       'away_corner', 'home_pass', 'away_pass', 'home_duel', 'away_duel',
       'home_foul', 'away_foul', 'home_save attempt', 'away_save attempt',
       'home_shot', 'away_shot', 'home_shot_on_target', 'away_shot_on_target',
       'home_free_kick', 'away_free_kick', 'home_odds', 'away_odds',
       'home_possession', 'away_possession', 'home_degree', 'away_degree',
       'home_diameter', 'away_diameter', 'home_bet_cent', 'away_bet_cent',
       'home_entropy', 'away_entropy'],
      dtype='object')

A bajnokságok első 25 fordulóján tanítjuk a modellünket, és az utolsó 13 fordulón (a német Bundesliga esetében az utolsó 9-en) pedig teszteljük.

In [29]:
train_df=matches[matches['gameweek']<=25]
test_df=matches[matches['gameweek']>25]

In [30]:
train_df

,home,away,home_goal,away_goal,gameweek,home_corner,away_corner,home_pass,away_pass,home_duel,...,home_possession,away_possession,home_degree,away_degree,home_diameter,away_diameter,home_bet_cent,away_bet_cent,home_entropy,away_entropy
130,Lazio,Verona,2,0,25,11,2,480,354,261,...,0.58,0.42,35.091,23.909,2,2,0.016,0.023,0.035,0.059
131,AC Milan,Sampdoria,1,0,25,10,1,523,455,188,...,0.53,0.47,39.909,35.273,2,2,0.027,0.018,0.035,0.036
132,Atalanta,Fiorentina,1,1,25,2,5,461,352,261,...,0.57,0.43,33.364,25.273,2,2,0.024,0.022,0.043,0.057
133,Benevento,Crotone,3,2,25,7,4,471,277,254,...,0.61,0.39,35.636,17.273,2,3,0.019,0.036,0.035,0.100
134,Bologna,Sassuolo,2,1,25,2,9,514,373,222,...,0.57,0.43,40.182,24.182,2,2,0.026,0.032,0.034,0.066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1820,Tottenham,Bournemouth,1,0,8,9,5,667,225,205,...,0.72,0.28,53.091,15.455,2,3,0.017,0.041,0.021,0.118
1821,Tottenham,Swansea,0,0,5,11,1,753,220,177,...,0.75,0.25,60.636,13.909,2,3,0.013,0.037,0.018,0.130
1822,Tottenham,Burnley,1,1,3,10,7,573,276,179,...,0.66,0.34,44.909,18.545,2,2,0.016,0.026,0.026,0.086
1823,Tottenham,Chelsea,1,2,2,14,3,569,249,223,...,0.68,0.32,43.545,17.091,2,2,0.009,0.030,0.025,0.098


In [48]:
test_df

,home,away,home_goal,away_goal,gameweek,home_corner,away_corner,home_pass,away_pass,home_duel,...,home_possession,away_possession,home_degree,away_degree,home_diameter,away_diameter,home_bet_cent,away_bet_cent,home_entropy,away_entropy
0,Lazio,Inter,2,3,38,10,3,302,492,228,...,0.40,0.60,22.000,37.545,2,2,0.027,0.019,0.072,0.032
1,Sassuolo,AS Roma,0,1,38,5,9,337,500,206,...,0.42,0.58,23.182,36.909,2,2,0.024,0.011,0.066,0.032
2,Cagliari,Atalanta,1,0,38,5,11,232,587,235,...,0.31,0.69,15.545,45.909,2,2,0.037,0.019,0.111,0.027
3,Chievo,Benevento,1,0,38,2,4,198,721,221,...,0.24,0.76,12.818,55.364,2,2,0.042,0.025,0.144,0.021
4,Udinese,Bologna,1,0,38,10,2,327,518,207,...,0.41,0.59,24.636,39.636,3,2,0.033,0.019,0.064,0.032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1806,Tottenham,Watford,2,0,36,4,0,548,322,209,...,0.61,0.39,41.545,22.545,2,2,0.016,0.026,0.029,0.067
1807,Tottenham,Manchester City,1,3,34,5,7,443,502,190,...,0.48,0.52,31.636,39.364,2,2,0.023,0.010,0.043,0.028
1808,Tottenham,Huddersfield,2,0,29,8,1,785,256,238,...,0.74,0.26,61.273,15.091,2,2,0.016,0.035,0.018,0.119
1809,Tottenham,Arsenal,1,0,27,10,2,488,488,218,...,0.50,0.50,36.091,36.455,2,2,0.022,0.020,0.037,0.035


In [35]:
home_attributes=[column for column in list(train_df) if 'home' in column]
away_attributes=[column for column in list(train_df) if 'away' in column]

In [39]:
melted = []

for _, row in train_df.iterrows():
    home_team = [row[attr] for attr in home_attributes] + [1]
    away_team = [row[attr] for attr in away_attributes] + [0]
    melted.append(home_team)
    melted.append(away_team)

print(melted[0])

['Lazio', 2, 11, 480, 261, 7, 0, 33, 10, 12, 1.19, 0.58, 35.091, 2, 0.016, 0.035, 1]


In [49]:
features = ['team', 'goal', 'corner', 'pass', 'duel', 'foul', 'save attempt', 'shot', 'shot_on_target', 'free_kick', 'odds', 'possession', 'degr', 'diameter',
            'bet_cent', 'entropy', 'is_at_home']

In [51]:
melted_df = pd.DataFrame(melted, columns = features)
melted_df.head(6)

,team,goal,corner,pass,duel,foul,save attempt,shot,shot_on_target,free_kick,odds,possession,degr,diameter,bet_cent,entropy,is_at_home
0,Lazio,2,11,480,261,7,0,33,10,12,1.19,0.58,35.091,2,0.016,0.035,1
1,Verona,0,2,354,261,11,11,5,0,10,15.86,0.42,23.909,2,0.023,0.059,0
2,AC Milan,1,10,523,188,13,2,17,6,16,1.65,0.53,39.909,2,0.027,0.035,1
3,Sampdoria,0,1,455,188,15,8,8,1,16,5.45,0.47,35.273,2,0.018,0.036,0
4,Atalanta,1,2,461,261,10,2,7,3,18,2.36,0.57,33.364,2,0.024,0.043,1
5,Fiorentina,1,5,352,261,19,3,15,2,12,3.21,0.43,25.273,2,0.022,0.057,0


In [52]:
# ezekre a statokra nem tanulunk rá
features_to_drop=['team', 'goal']

In [53]:
X = melted_df.drop(columns = features_to_drop).values
y = melted_df['goal'].values
reg = GradientBoostingRegressor(loss = 'ls', max_depth = 10)
reg.fit(X, y)
reg

GradientBoostingRegressor(loss='ls', max_depth=10)

In [55]:
reg.predict([[11,420,188,4,4,30,12,14,1.24,0.54,41.234,2,0.03,0.035,1]])

array([4.75792242])